# Space-geodetic data

_The following notebook uses a proprietary library called Where. At the moment it is not available outside Kartverket._

There are many different data formats and data types one runs into when working with satellite and space geodesy data. As part of the data flow, `where` is able to read many of these. However, these data can be taken advantage of also outside the `where`-pipeline.

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

## Parsers

Parsers are `where`-functions for reading data. A parser can typically read a data file and return data in a structured dictionary. The following parsers are currently available in the `where`-library.

In [ ]:
from where import parsers
parsers.names()

As an example, let us try to read a Rinex 2-file:

In [ ]:
!head examples/nyal3060.09o

The following returns the data as a _dictionary_:

In [ ]:
nyal = parsers.parse('rinex2_obs', file_path='examples/nyal3060.09o', sampling_rate=300).data

In [ ]:
nyal.keys()

In [ ]:
nyal['time']

In [ ]:
nyal['obs'].keys()

`where` uses its own `Dataset`-format for doing analyses, but when using `where` as a library it is often easier to store the data in a `pandas` dataframe (this is planned to be supported directly):

In [ ]:
import pandas as pd
df = pd.DataFrame(dict(time=nyal['time'], satellite=nyal['text']['satellite'],
                       C1=nyal['obs']['C1']))
df.head()

In [ ]:
df.time = pd.to_datetime(df.time)
df = df.set_index(['time', 'satellite'])
df = df.unstack()
df.columns = df.columns.droplevel()
df.head()

In [ ]:
df.G08.plot()

## Apriori data

The parsers are quite low level, only reading one file and returning the data in a fairly raw form. Another layer is introduced for many types of data inside the `apriori`-library:

In [ ]:
from where import apriori
apriori.names()

### Ephemerides

One example is the `ephemerides` apriori-data.

In [ ]:
from where.lib.time import Time
y2016 = Time(range(57388,57754), format='mjd', scale='utc')

In [ ]:
eph = apriori.get('ephemerides', time=y2016)
eph.names

In [ ]:
earth = eph.pos_bcrs('earth')
venus = eph.pos_bcrs('venus')
mars = eph.pos_bcrs('mars', time=y1995_2016)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*earth.T, color='blue')
ax.scatter(*venus.T, color='orange')
ax.scatter(*mars.T, color='red')
ax;

### Reference frames

Another example is reference frames

In [ ]:
import where
from where.lib import files
where.set_config(2015, 8, 4, 'vlbi')
files.use_filekey_suffix('vlbi')

y1995_2016 = Time(range(49718,57754), format='mjd', scale='utc', set_deltas=False)
y1995_2016.iso

In [ ]:
itrf2014 = apriori.get('trf', time=y1995_2016, reference_frame='itrf_snx:2014')
itrf2008 = apriori.get('trf', time=y1995_2016, reference_frame='itrf_ssc:2008')

In [ ]:
itrf2008[('vlbi', '7331')]

In [ ]:
itrf2014[('vlbi', '7331')]

In [ ]:
itrf2008.pos(('vlbi', '7331'), time=1)

In [ ]:
itrf2014.pos(('vlbi', '7331'))

In [ ]:
compare_z = np.vstack((itrf2008.pos(('vlbi', '7331'))[:, 2], itrf2014.pos(('vlbi', '7331'))[:, 2])).T
plt.plot(y1995_2016.datetime, compare_z);

In [ ]:
plt.plot(y1995_2016.datetime, itrf2014.pos(('vlbi', '7345'))[:, 1]);